# Naive Bayers

In [1]:
from sklearn.model_selection import train_test_split
##from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import sklearn

In [2]:
# Função para avaliação dos modelos
def avalia_modelo(clf, X, y):
    resultados = sklearn.model_selection.cross_val_predict(clf, X, y, cv=5)
    # print( pd.crosstab(y, resultados, rownames=['Real'], colnames=['Predito'], margins=True))
    return np.mean(sklearn.model_selection.cross_val_score(clf, X, y, cv=5))

In [3]:
def doIniciaResult(tamanho):
    # Iniciar um array com vazios
    df_resultado     = [ [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] ]
    for i in range( tamanho ):
        df_resultado.append( [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ] )

    #df_resultado = [] #pd.DataFrame(columns = colunas)
    colunas = [ 'Remetente','registros1','precisao1','registros2','precisao2','registros3','precisao3','registros4','precisao4' ]
    return df_resultado

In [4]:
def doFornecedorCut(dtNotas, fornecedores, tamanho = 1000):
    # Obtem lista de fornecedores com mais de 10000 itens
    fornecedores = dtNotas['Remetente'].value_counts()
    mask = fornecedores > tamanho
    fornecedores = list(fornecedores[mask].index)
    return fornecedores

In [5]:
def doPrint(linha, coluna, col0=-1, col1 = -1, col2 = -1, col3 = -1, col4=-1):
    print ("{}:{} - [{}][{}][{}][{}][{}]".format(str(linha), str(coluna), str(col0), str(col1), str(col2), str(col3), str(col4)))

In [6]:
# Processa o Naives
def doProcNaives(dfNFiscais, fornecedores):
  
	# Iniciar o Resutlado
	df_resultado = doIniciaResult(len( fornecedores ))
	linha  = 0
	
	# Cria um Pipeline - Classificador Composto
	# vectorizer => transformer => classifier 
	text_clf = Pipeline([('counts' , CountVectorizer() ), ('classifier', MultinomialNB()),])

	# registra resultados
	df_resultado[linha][0] = 'Todos'
	df_resultado[linha][1] = dfNFiscais.shape[0]
	df_resultado[linha][2] = avalia_modelo(text_clf, dfNFiscais['NovaDescricao'], dfNFiscais['Categoria'])

	doPrint(linha, 1, col0=df_resultado[linha][0], col1=df_resultado[linha][1], col2=df_resultado[linha][2])
	# Cria um Pipeline - Classificador Composto
	# vectorizer => transformer => classifier 
	text_clf = Pipeline([('counts' , CountVectorizer() ), ('tfidf', TfidfTransformer()), ('classifier', MultinomialNB()),])

	# registra resultados
	df_resultado[linha][3] = dfNFiscais.shape[0]
	df_resultado[linha][4] = avalia_modelo(text_clf, dfNFiscais['NovaDescricao'], dfNFiscais['Categoria'])    

	doPrint(linha, 3, col0=df_resultado[linha][0], col3=df_resultado[linha][3], col4=df_resultado[linha][4])

	# calcula precisão para fornecedores
	for fornecedor in fornecedores:
		linha += 1
		
		# obtem dados do fornecedor
		mask = dfNFiscais["Remetente"] == fornecedor
		dados_fornecedor = dfNFiscais[ mask ]

		# Cria um Pipeline - Classificador Composto
		# vectorizer => transformer => classifier 
		text_clf = Pipeline([('counts' , CountVectorizer() ),('classifier'  , MultinomialNB()),])

		# registra resultados
		df_resultado[linha][0] = fornecedor
		df_resultado[linha][1] = dados_fornecedor.shape[0]
		df_resultado[linha][2] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], dados_fornecedor['Categoria'])
		doPrint(linha, 1, col0=df_resultado[linha][0], col1=df_resultado[linha][1], col2=df_resultado[linha][2])

		# Cria um Pipeline - Classificador Composto
		# vectorizer => transformer => classifier 
		text_clf = Pipeline([('counts' , CountVectorizer() ),('tfidf', TfidfTransformer()),('classifier'  , MultinomialNB()),])

		# registra resultados
		df_resultado[linha][3] = dados_fornecedor.shape[0]
		df_resultado[linha][4] = avalia_modelo(text_clf, dados_fornecedor['NovaDescricao'], dados_fornecedor['Categoria'])         
		doPrint(linha, 3, col0=df_resultado[linha][0], col3=df_resultado[linha][3], col4=df_resultado[linha][4])

	return df_resultado
    

In [ ]:
# create the transform
vectorizer = CountVectorizer()

text = dfNFiscais['NovaDescricao']
# tokenize and build vocab
vectorizer.fit(text)

# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())


In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
palavras = list(vectorizer.vocabulary_)
palavras.sort()
len(palavras)

In [ ]:
palavras

In [ ]:
df_palavras = pd.DataFrame(data = palavras)


In [ ]:
df_palavras.to_csv('palavras.cvs', sep=";", index=False )

In [ ]:
vocabulário = list( vectorizer.vocabulary_ )
vocabulário

In [ ]:
vocabulário.sort()
vocabulário

In [7]:
def doNaiveBayers(dfNFiscais):    
    fornecedores = doFornecedorCut(dfNFiscais, 1000)
    resultado = doProcNaives(dfNFiscais, fornecedores)
    
    return resultado
    

In [9]:
# SANDBOX
import import_ipynb
import numpy as np
from carga import doCarga

diretorio = '..\\dados\\'
dfNFiscais = doCarga(diretorio, opcao=3) 

fornecedores = doFornecedorCut(dfNFiscais, 1000)
resultado = doProcNaives(dfNFiscais, fornecedores)


#print ("Precisão: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
dfNFiscais.count

0:1 - [Todos][99146][0.9290756772217893][-1][-1]
0:3 - [Todos][-1][-1][99146][0.9356918257251797]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


1:1 - [Fornecedor 063][8517][0.999765189295885][-1][-1]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


1:3 - [Fornecedor 063][-1][-1][8517][0.9998826291079812]
2:1 - [Fornecedor 011][8059][0.9812626591685882][-1][-1]
2:3 - [Fornecedor 011][-1][-1][8059][0.9888322066191954]
3:1 - [Fornecedor 121][7319][0.9681585670584807][-1][-1]
3:3 - [Fornecedor 121][-1][-1][7319][0.9648852944582021]


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\s

32:1 - [Fornecedor 052][1056][0.9914870786014486][-1][-1]
32:3 - [Fornecedor 052][-1][-1][1056][0.9810694804614147]
33:1 - [Fornecedor 005][1051][0.9781494019408712][-1][-1]
33:3 - [Fornecedor 005][-1][-1][1051][0.9591243511622659]
34:1 - [Fornecedor 038][1023][0.9335915829746533][-1][-1]
34:3 - [Fornecedor 038][-1][-1][1023][0.9238211382113821]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


35:1 - [Fornecedor 020][1009][0.9871287128712872][-1][-1]
35:3 - [Fornecedor 020][-1][-1][1009][0.9742327964139698]


<bound method DataFrame.count of             Remetente   Categoria                      NovaDescricao
0      Fornecedor 001  30.0simsim      CACHACA SAO FRANCISCO 12X970M
1      Fornecedor 001  30.0simsim            CACHACA YPIOCA EMP.OURO
2      Fornecedor 001  30.0simsim                CACHACA YPIOCA OURO
3      Fornecedor 001  30.0simsim               CACHACA YPIOCA PRATA
4      Fornecedor 002  30.0simsim  BB CACHACA SAGATIBA PURA  (12) GF
...               ...         ...                                ...
99995  Fornecedor 047  38.0simnão      SHAMP.INF.BEBE NATUREZA SUAVE
99996  Fornecedor 047  38.0simnão      SHAMP.INF.BEBE NATUREZA SUAVE
99997  Fornecedor 047  38.0simnão        SHAMP.INF.CARROS II MCQUEEN
99998  Fornecedor 047  38.0simnão      SHAMP.INF.CARROS II MCQUEENNC
99999  Fornecedor 047  38.0simnão      SHAMP.INF.CARROS II MCQUEENNC

[99146 rows x 3 columns]>

In [ ]:
# RECORTA FONTE
"""
import import_ipynb
import numpy as np
from carga import doCarga

diretorio = '..\\dados\\'
dfNFiscais = doCarga(diretorio, opcao=4) 

dfmini = dfNFiscais.iloc[0:1000]
dfmini.describe
dfmini.to_pickle(diretorio + "df_Mini.pkl")
"""